This notebook generates a map with a circle for each location where EU/Schengen state visas are issued. 
* The dataset contains one row per origin country and consulate, summing up the total number of short-stay visas applied, issued and not issued. 
* The colour of the circle indicates the restrictiveness of the visa issuing practice (the share of visas issued relative to the visas issued and not issued). 
* The size of the circle indicates how many visas are received. 

In [1]:
import pandas as pd
df_cities = pd.read_csv('../data/gold/cities.csv')
df_visas = pd.read_csv('../data/silver/schengen-visa-2020.csv')

In [2]:
df = pd.merge(df_visas, df_cities[["origin_country", "origin_consulate", "latitude", "longitude"]], how="inner", on=["origin_country", "origin_consulate"])

In [3]:
df = df[df["visa_refusal_rate"].notna()]
df = df[df['latitude'].notna()]

In [4]:
df2 = df.groupby(["origin_country","origin_consulate","latitude","longitude"])[["visas_applied", "visas_issued", "visas_not_issued"]].apply(lambda x: x.sum()).reset_index()
df2["visa_refusal_rate"] = df2["visas_not_issued"] / (df2["visas_issued"] + df2["visas_not_issued"])

In [5]:
df2["visas_applied_scaled"] = df2.apply(lambda x: (x["visas_applied"] - df2['visas_applied'].min()) / (df2['visas_applied'].max() - df2['visas_applied'].min()), axis = 1)

In [6]:
import folium
import branca.colormap as cm

def popup_producer(origin_country, origin_consulate, visa_refusal_rate,visas_applied):
    return origin_country + ', ' + origin_consulate + '\n' + ':\nref.rate: {:.0%}'.format(visa_refusal_rate) + '\nApp.: {0:.0f}'.format(visas_applied)

colormap = cm.LinearColormap(colors=['lightgreen','red'])
colormap.caption = 'Visa refusal rate'

m = folium.Map(location=[54, 15], tiles='Stamen Toner', zoom_start=3)
m.add_child(colormap)

for idx, row in df2.iterrows():
    m.add_child(folium.Circle(
        location=[row['latitude'], row['longitude']], 
        radius=row['visas_applied_scaled'] * 1000000,
        fill = True,
        popup=popup_producer(row['origin_country'],row['origin_consulate'],row['visa_refusal_rate'],row['visas_applied']),
        color=colormap(row['visa_refusal_rate'])))
m.save('../html/map-visa-statistics-2020.html')
m